# Creating a multi-temporal datacube of Sentinel-2 data

In [1]:
import openeo
openeo.client_version()

'0.19.0'

This notebook shows how to create a datacube from Sentinel-2 data for further processing. It follows the more generic example available here: https://github.com/Open-EO/openeo-community-examples/blob/main/python/BasicSentinelMerge/sentinel_merge.ipynb. In this notebook, we create monthly composites from Sentinel-2 data, and apply linear interpolation to avoid gaps. Specific methods may of course require different cloud masking and preprocessing options.

In [2]:
c=openeo.connect("openeo.vito.be")#("openeo.cloud")
c.authenticate_oidc()


Authenticated using refresh token.


<Connection to 'https://openeo.vito.be/openeo/1.1/' with OidcBearerAuth>

In [14]:
# Define the inputs
temporal_extent = ["2020-01-01", "2022-12-31"] # temporal extent 
s2_bands = ["B02", "B03", "B04", "B08", "B11", "B12", "SCL"] # extracted bands
max_cloud_cover=75 # only process images with a cloud cover lower than this threshold
out_folder = './' # folder where the data will be stored
composite_period = 'month' # composite period
composite_method= 'median' # composite method
spatial_extent = {'west': 399960, 'east': 400560, 'south': 4190220, 'north': 4190820, 'crs': 'epsg:32630'} # AOI


To use the data collection, a user must use the correct backend with the data collection. Then using load_collection, they can specify bands, temporal extent (i.e. interested time interval), the maximum cloud cover and even spatial extent. More information can be found *[here](https://open-eo.github.io/openeo-python-client/datacube_construction.html#)* and *[here](https://open-eo.github.io/openeo-python-client/api.html#openeo.rest.connection.Connection.load_collection)*.

Next, we apply cloud masking using dilation and erosion of the SCL layer, and composite the data to the desired composite period.

In [8]:

# load collection
sentinel2 = c.load_collection(
    "SENTINEL2_L2A",
    temporal_extent = temporal_extent,
    bands = s2_bands,
    max_cloud_cover=max_cloud_cover
)

# cloud masking based on the SCL layer
sentinel2 = sentinel2.process(
            "mask_scl_dilation",
            data=sentinel2,
            scl_band_name="SCL",
            kernel1_size=17, kernel2_size=77,
            mask1_values=[2, 4, 5, 6, 7],
            mask2_values=[3, 8, 9, 10, 11],
            erosion_kernel_size=3)

# composite the data using the median value per month
sentinel2 = sentinel2.aggregate_temporal_period(composite_period, reducer= composite_method)\
    .apply_dimension(dimension="t", process="array_interpolate_linear")

/home/wdkeers/miniconda3/envs/eo42/lib/python3.9/site-packages/openeo/rest/connection.py:1087: UserWarning: SENTINEL2_L2A property filtering with properties that are undefined in the collection metadata (summaries): eo:cloud_cover.
  return DataCube.load_collection(


We will scale the data with a factor 10000 using a UDF. More information on UDF can be found *[here](https://open-eo.github.io/openeo-python-client/udf.html)*.

In [9]:
# scale the data using a user defined function
my_udf = openeo.UDF("""
from openeo.udf import XarrayDataCube
from openeo.udf.debug import inspect

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    array = cube.get_array()
    inspect(array,level="ERROR",message="inspecting input cube")
    array.values = 0.0001 * array.values
    return cube
""")

sentinel2 = sentinel2.apply_neighborhood(my_udf, size=[
        {'dimension': 'x', 'value': 112, 'unit': 'px'},
        {'dimension': 'y', 'value': 112, 'unit': 'px'}
    ], overlap=[
        {'dimension': 'x', 'value': 8, 'unit': 'px'},
        {'dimension': 'y', 'value': 8, 'unit': 'px'}
    ])

Finally, we crop the data to the pre-defined spatial extent and download the processed data.

In [10]:
job=sentinel2.filter_bbox(spatial_extent).execute_batch()
results = job.get_results()
results.download_files(out_folder)

0:00:00 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': send 'start'
0:00:20 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:00:25 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:00:32 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:00:40 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:00:50 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:01:02 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:01:19 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:01:38 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': queued (progress N/A)
0:02:02 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': running (progress N/A)
0:02:34 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': running (progress N/A)
0:03:11 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': running (progress N/A)
0:03:58 Job 'vito-j-622703ff3b4f4fc1abdfe6557fea192f': running (progress N/A)
0:04

[PosixPath('openEO_2020-01-01Z.tif'),
 PosixPath('openEO_2020-02-01Z.tif'),
 PosixPath('openEO_2020-03-01Z.tif'),
 PosixPath('openEO_2020-04-01Z.tif'),
 PosixPath('openEO_2020-05-01Z.tif'),
 PosixPath('openEO_2020-06-01Z.tif'),
 PosixPath('openEO_2020-07-01Z.tif'),
 PosixPath('openEO_2020-08-01Z.tif'),
 PosixPath('openEO_2020-09-01Z.tif'),
 PosixPath('openEO_2020-10-01Z.tif'),
 PosixPath('openEO_2020-11-01Z.tif'),
 PosixPath('openEO_2020-12-01Z.tif'),
 PosixPath('openEO_2021-01-01Z.tif'),
 PosixPath('openEO_2021-02-01Z.tif'),
 PosixPath('openEO_2021-03-01Z.tif'),
 PosixPath('openEO_2021-04-01Z.tif'),
 PosixPath('openEO_2021-05-01Z.tif'),
 PosixPath('openEO_2021-06-01Z.tif'),
 PosixPath('openEO_2021-07-01Z.tif'),
 PosixPath('openEO_2021-08-01Z.tif'),
 PosixPath('openEO_2021-09-01Z.tif'),
 PosixPath('openEO_2021-10-01Z.tif'),
 PosixPath('openEO_2021-11-01Z.tif'),
 PosixPath('openEO_2021-12-01Z.tif'),
 PosixPath('openEO_2022-01-01Z.tif'),
 PosixPath('openEO_2022-02-01Z.tif'),
 PosixPath('